In [56]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [2]:
%%capture
!pip install xgboost

In [57]:
pd.set_option("display.max_columns", None)
df = pd.read_csv('k2pandc_2025.10.03_17.22.31 (1).csv')
df = df[df["disposition"].isin(["CONFIRMED", "FALSE POSITIVE", "CANDIDATE"])]
df.head()

,loc_rowid,pl_name,hostname,default_flag,disposition,disp_refname,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radj,pl_radjerr1,pl_radjerr2,pl_radjlim,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassj,pl_bmassjerr1,pl_bmassjerr2,pl_bmassjlim,pl_bmassprov,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,ttv_flag,st_refname,st_spectype,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_metratio,st_logg,st_loggerr1,st_loggerr2,st_logglim,sy_refname,rastr,ra,decstr,dec,sy_dist,sy_disterr1,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
0,1,BD+20 594 b,BD+20 594,0,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,K2,Published Confirmed,0,<a refstr=MAYO_ET_AL__2018 href=https://ui.ads...,41.688644,0.003353,-0.003419,0.0,NaN,NaN,NaN,NaN,2.355454,0.310000,-0.166773,0.0,0.210139,0.027656,-0.014878,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,<a refstr=MAYO_ET_AL__2018 href=https://ui.ads...,NaN,5703.00,50.00,-50.00,0.0,0.956122,0.098585,-0.054675,0.0,0.963861,0.029568,-0.032166,0.0,-0.06,0.080,-0.080,0.0,[Fe/H],4.38000,0.100000,-0.100000,0.0,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,03h34m36.27s,53.651123,+20d35m56.47s,20.599021,179.4610,1.2570,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.8644,0.000249,-0.000249,2018-04-25,2018-03,2018-02-15
1,2,BD+20 594 b,BD+20 594,0,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,K2,Published Confirmed,0,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,41.685500,0.003000,-0.003100,0.0,0.241,0.019,-0.017,0.0,2.230000,0.140000,-0.110000,0.0,0.198947,0.012490,-0.009814,0.0,16.3000,6.0000,-6.1000,0.0,0.051285,0.018878,-0.019193,0.0,Mass,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,546.0,19.0,-18.0,0.0,0,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,G,5766.00,99.00,-99.00,0.0,0.928000,0.055000,-0.040000,0.0,0.961000,0.032000,-0.029000,0.0,-0.15,0.050,-0.050,0.0,[Fe/H],4.50000,0.080000,-0.080000,0.0,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,03h34m36.27s,53.651123,+20d35m56.47s,20.599021,179.4610,1.2570,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.8644,0.000249,-0.000249,2018-04-25,2016-10,2016-07-28
2,3,BD+20 594 b,BD+20 594,1,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,K2,Published Confirmed,0,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,41.685500,0.003000,-0.003000,0.0,NaN,NaN,NaN,NaN,2.578070,0.112090,-0.112090,0.0,0.230000,0.010000,-0.010000,0.0,22.2481,9.5349,-9.5349,0.0,0.070000,0.030000,-0.030000,0.0,Mass,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,NaN,5766.00,99.00,-99.00,0.0,1.080000,0.060000,-0.060000,0.0,1.670000,0.400000,-0.400000,0.0,-0.15,NaN,NaN,0.0,[Fe/H],4.50000,0.080000,-0.080000,0.0,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,03h34m36.27s,53.651123,+20d35m56.47s,20.599021,179.4610,1.2570,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.8644,0.000249,-0.000249,2018-04-25,2017-03,2018-04-26
3,4,EPIC 201111557.01,EPIC 201111557,1,CANDIDATE,Livingston et al. 2018,1,0,Transit,2018,K2,Published Candidate,0,<a refstr=LIVINGSTON_ET_AL__2018 href=https://...,2.301830,0.000280,-0.000300,0.0,NaN,NaN,NaN,NaN,1.120000,0.110000,-0.080000,0.0,0.099900,0.009800,-0.007000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1054.0,55.0,-55.0,0.0,0,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,NaN,4616.52,82.36,-115.56,0.0,0.762602,0.053651,-0.037585,0.0,0.730000,0.084105,-0.081044,0.0,-0.03,0.034,-0.034,0.0,[M/H],4.53679,0.075486,-0.0905

In [58]:
value = df['disposition'].value_counts()
value

disposition
CONFIRMED         2315
CANDIDATE         1374
FALSE POSITIVE     293
Name: count, dtype: int64

In [59]:
df.drop(['loc_rowid','pl_name','hostname','disp_refname','disc_year',
'pl_refname','st_refname','sy_refname','rastr','decstr','rowupdate','pl_pubdate','releasedate'],axis=1,inplace=True)
df = df.loc[:, df.notnull().sum() >= 1000]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3982 entries, 0 to 4003
Data columns (total 56 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   default_flag     3982 non-null   int64  
 1   disposition      3982 non-null   object 
 2   sy_snum          3982 non-null   int64  
 3   sy_pnum          3982 non-null   int64  
 4   discoverymethod  3982 non-null   object 
 5   disc_facility    3982 non-null   object 
 6   soltype          3982 non-null   object 
 7   pl_controv_flag  3982 non-null   int64  
 8   pl_orbper        3938 non-null   float64
 9   pl_orbpererr1    3054 non-null   float64
 10  pl_orbpererr2    3054 non-null   float64
 11  pl_orbperlim     3938 non-null   float64
 12  pl_rade          3155 non-null   float64
 13  pl_radeerr1      2872 non-null   float64
 14  pl_radeerr2      2872 non-null   float64
 15  pl_radelim       3155 non-null   float64
 16  pl_radj          3155 non-null   float64
 17  pl_radjerr1      28

In [60]:
print(df['discoverymethod'].unique())
print(df['soltype'].unique())
print(df['st_metratio'].unique())
print(df['disc_facility'].unique())
print(df['disposition'].unique())

['Transit' 'Radial Velocity' 'Microlensing']
['Published Confirmed' 'Published Candidate' 'TESS Project Candidate']
['[Fe/H]' '[M/H]' nan '[m/H]']
['K2' 'HATNet' 'HATSouth' 'Multiple Observatories'
 'Roque de los Muchachos Observatory'
 'CHaracterising ExOPlanets Satellite (CHEOPS)' 'La Silla Observatory'
 'Qatar' 'SuperWASP-South' 'SuperWASP']
['CONFIRMED' 'CANDIDATE' 'FALSE POSITIVE']


In [61]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

# Aplicar no label
df["label"] = encoder.fit_transform(df["disposition"])
df["discoverymethod_encoded"] = encoder.fit_transform(df["discoverymethod"])
df["soltype_encoded"] = encoder.fit_transform(df["soltype"])
df["disc_facility_encoded"] = encoder.fit_transform(df["disc_facility"])

# Fill NaN with placeholder
df["st_metratio"] = df["st_metratio"].fillna("missing")
df["st_metratio_encoded"] = encoder.fit_transform(df["st_metratio"])


df.drop(['st_metratio','disposition','discoverymethod','soltype','disc_facility'],axis=1,inplace=True)
df.head()

,default_flag,sy_snum,sy_pnum,pl_controv_flag,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radj,pl_radjerr1,pl_radjerr2,pl_radjlim,ttv_flag,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_logg,st_loggerr1,st_loggerr2,st_logglim,ra,dec,sy_dist,sy_disterr1,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,label,discoverymethod_encoded,soltype_encoded,disc_facility_encoded,st_metratio_encoded
0,0,1,1,0,41.688644,0.003353,-0.003419,0.0,2.355454,0.310000,-0.166773,0.0,0.210139,0.027656,-0.014878,0.0,0,5703.00,50.00,-50.00,0.0,0.956122,0.098585,-0.054675,0.0,0.963861,0.029568,-0.032166,0.0,-0.06,0.080,-0.080,0.0,4.38000,0.100000,-0.100000,0.0,53.651123,20.599021,179.4610,1.2570,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.8644,0.000249,-0.000249,1,2,1,3,0
1,0,1,1,0,41.685500,0.003000,-0.003100,0.0,2.230000,0.140000,-0.110000,0.0,0.198947,0.012490,-0.009814,0.0,0,5766.00,99.00,-99.00,0.0,0.928000,0.055000,-0.040000,0.0,0.961000,0.032000,-0.029000,0.0,-0.15,0.050,-0.050,0.0,4.50000,0.080000,-0.080000,0.0,53.651123,20.599021,179.4610,1.2570,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.8644,0.000249,-0.000249,1,2,1,3,0
2,1,1,1,0,41.685500,0.003000,-0.003000,0.0,2.578070,0.112090,-0.112090,0.0,0.230000,0.010000,-0.010000,0.0,0,5766.00,99.00,-99.00,0.0,1.080000,0.060000,-0.060000,0.0,1.670000,0.400000,-0.400000,0.0,-0.15,NaN,NaN,0.0,4.50000,0.080000,-0.080000,0.0,53.651123,20.599021,179.4610,1.2570,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.8644,0.000249,-0.000249,1,2,1,3,0
3,1,1,0,0,2.301830,0.000280,-0.000300,0.0,1.120000,0.110000,-0.080000,0.0,0.099900,0.009800,-0.007000,0.0,0,4616.52,82.36,-115.56,0.0,0.762602,0.053651,-0.037585,0.0,0.730000,0.084105,-0.081044,0.0,-0.03,0.034,-0.034,0.0,4.53679,0.075486,-0.090593,0.0,183.846245,-6.268329,97.1795,0.4642,-0.4598,11.727,0.046,-0.046,9.220,0.019,-0.019,11.3995,0.001307,-0.001307,0,2,0,3,1
4,0,1,0,0,2.302368,0.000105,-0.000103,0.0,1.312588,0.524177,-0.120814,0.0,0.120000,0.050000,-0.010000,0.0,0,4720.00,50.00,-50.00,0.0,0.710995,0.018609,-0.020398,0.0,NaN,NaN,NaN,NaN,-0.06,0.080,-0.080,0.0,4.50000,0.100000,-0.100000,0.0,183.846245,-6.268329,97.1795,0.4642,-0.4598,11.727,0.046,-0.046,9.220,0.019,-0.019,11.3995,0.001307,-0.001307,0,2,0,3,0


In [62]:
for col in ["discoverymethod_encoded", "disc_facility_encoded", "soltype_encoded"]:
    if df[col].isnull().any():
        df.fillna({col: df[col].mode()[0]}, inplace=True)
df = df.fillna(df.mean())
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3982 entries, 0 to 4003
Data columns (total 56 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   default_flag             3982 non-null   int64  
 1   sy_snum                  3982 non-null   int64  
 2   sy_pnum                  3982 non-null   int64  
 3   pl_controv_flag          3982 non-null   int64  
 4   pl_orbper                3982 non-null   float64
 5   pl_orbpererr1            3982 non-null   float64
 6   pl_orbpererr2            3982 non-null   float64
 7   pl_orbperlim             3982 non-null   float64
 8   pl_rade                  3982 non-null   float64
 9   pl_radeerr1              3982 non-null   float64
 10  pl_radeerr2              3982 non-null   float64
 11  pl_radelim               3982 non-null   float64
 12  pl_radj                  3982 non-null   float64
 13  pl_radjerr1              3982 non-null   float64
 14  pl_radjerr2              3982

In [63]:
import numpy as np

# 1. Relações estrela–planeta
# Razão raio do planeta pelo raio da estrela
df["pl_rade_st_ratio"] = df["pl_rade"] / df["st_rad"]

# Densidade aproximada (massa/raio³) para estrela e planeta
# (valores em unidades relativas, já que dados vêm em raio solar/jupiteriano/terrestre)
df["st_density_est"] = df["st_mass"] / (df["st_rad"] ** 3)
df["pl_density_est"] = np.where(df["pl_rade"] > 0, df["pl_radj"] / (df["pl_rade"] ** 3), np.nan)

# Temperatura de equilíbrio planetária aproximada
# Usando 3ª Lei de Kepler: a ≈ ( (P² * M*)^(1/3) )
# P em dias → converter para segundos
G = 6.67430e-11  # constante gravitacional
M_sun = 1.98847e30  # massa do Sol em kg
R_sun = 6.9634e8    # raio do Sol em m

# Transformar unidades aproximadas (massa e raio em solares, período em dias)
P_sec = df["pl_orbper"] * 24 * 3600
M_star_kg = df["st_mass"] * M_sun
a_m = ((G * M_star_kg * (P_sec ** 2)) / (4 * (np.pi ** 2))) ** (1/3)

R_star_m = df["st_rad"] * R_sun
Teff = df["st_teff"]

df["pl_eq_temp"] = Teff * np.sqrt(R_star_m / (2 * a_m))

# 2. Magnitudes combinadas (cores estelares aproximadas)
df["vmag_minus_kmag"] = df["sy_vmag"] - df["sy_kmag"]
df["vmag_minus_gaiamag"] = df["sy_vmag"] - df["sy_gaiamag"]

# 3. Features de incerteza
df["pl_orbper_uncertainty"] = df["pl_orbpererr1"].abs() + df["pl_orbpererr2"].abs()
df["pl_rade_uncertainty"] = df["pl_radeerr1"].abs() + df["pl_radeerr2"].abs()
df["st_teff_uncertainty"] = df["st_tefferr1"].abs() + df["st_tefferr2"].abs()
df["st_rad_uncertainty"] = df["st_raderr1"].abs() + df["st_raderr2"].abs()
df["st_mass_uncertainty"] = df["st_masserr1"].abs() + df["st_masserr2"].abs()
df["sy_dist_uncertainty"] = df["sy_disterr1"].abs() + df["sy_disterr2"].abs()
df["sy_vmag_uncertainty"] = df["sy_vmagerr1"].abs() + df["sy_vmagerr2"].abs()
df["sy_kmag_uncertainty"] = df["sy_kmagerr1"].abs() + df["sy_kmagerr2"].abs()
df["sy_gaiamag_uncertainty"] = df["sy_gaiamagerr1"].abs() + df["sy_gaiamagerr2"].abs()

In [64]:
df['label'].value_counts()

label
1    2315
0    1374
2     293
Name: count, dtype: int64

In [65]:
value

disposition
CONFIRMED         2315
CANDIDATE         1374
FALSE POSITIVE     293
Name: count, dtype: int64

In [66]:
df_final = df
df_final = df.copy()
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3982 entries, 0 to 4003
Data columns (total 71 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   default_flag             3982 non-null   int64  
 1   sy_snum                  3982 non-null   int64  
 2   sy_pnum                  3982 non-null   int64  
 3   pl_controv_flag          3982 non-null   int64  
 4   pl_orbper                3982 non-null   float64
 5   pl_orbpererr1            3982 non-null   float64
 6   pl_orbpererr2            3982 non-null   float64
 7   pl_orbperlim             3982 non-null   float64
 8   pl_rade                  3982 non-null   float64
 9   pl_radeerr1              3982 non-null   float64
 10  pl_radeerr2              3982 non-null   float64
 11  pl_radelim               3982 non-null   float64
 12  pl_radj                  3982 non-null   float64
 13  pl_radjerr1              3982 non-null   float64
 14  pl_radjerr2              3982

In [67]:
df_final.shape

(3982, 71)

In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb
import joblib
from time import time

start = time()

# ==============================
# SPLITS E MODELO BASE
# ==============================

# Features e label
X = df_final.drop(['label'], axis=1)
y = df_final['label']

# Primeiro split: 70% treino, 30% restante
X_train_full, X_temp, y_train_full, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Segundo split: 20% teste, 10% blind validation
X_test, X_blind, y_test, y_blind = train_test_split(
    X_temp, y_temp, test_size=0.3333, stratify=y_temp, random_state=42
)

print(f"Train shape: {X_train_full.shape}")
print(f"Test shape: {X_test.shape}")
print(f"Blind validation shape: {X_blind.shape}")

# ==============================
# CROSS-VALIDATION
# ==============================

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_full, y_train_full), 1):
    print(f"\nFold {fold}")
    X_train, X_val = X_train_full.iloc[train_idx], X_train_full.iloc[val_idx]
    y_train, y_val = y_train_full.iloc[train_idx], y_train_full.iloc[val_idx]

    model = xgb.XGBClassifier(eval_metric='mlogloss', objective='multi:softprob')
    model.fit(X_train.values, y_train.values)

    y_pred_val = model.predict(X_val.values)
    acc = accuracy_score(y_val, y_pred_val)
    f1 = f1_score(y_val, y_pred_val, average='weighted')
    print(f"Validation Accuracy: {acc:.4f} | F1-score: {f1:.4f}")
    fold_results.append({'fold': fold, 'accuracy': acc, 'f1': f1})

acc_mean = np.mean([r['accuracy'] for r in fold_results])
f1_mean = np.mean([r['f1'] for r in fold_results])
print(f"\nMédia Validation Accuracy: {acc_mean:.4f}")
print(f"Média Validation F1-score: {f1_mean:.4f}")

# ==============================
# MODELO FINAL COMPLETO
# ==============================

final_model = xgb.XGBClassifier(eval_metric='mlogloss', objective='multi:softprob')
final_model.fit(X_train_full.values, y_train_full.values)

feat_imp = pd.Series(final_model.feature_importances_, index=X_train_full.columns).sort_values(ascending=False)
feat_imp.to_csv('summary/K2_complete_all_feature_importance.csv')
print("\nTop 10 Feature importances:")
print(feat_imp.head(10))

# Avaliação no teste
y_pred_test = final_model.predict(X_test.values)
print("\n--- Base Model Test Results ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_test, average='weighted'):.4f}")

# Avaliação na blind validation
y_pred_blind = final_model.predict(X_blind.values)
print("\n--- Base Model Blind Results ---")
print(f"Accuracy: {accuracy_score(y_blind, y_pred_blind):.4f}")
print(f"F1-score: {f1_score(y_blind, y_pred_blind, average='weighted'):.4f}")

#joblib.dump(final_model, 'xgb_final_model.pkl')

# ==============================
# TESTES COM 90%, 95%, 99% DAS FEATURES
# ==============================

thresholds = [0.90, 0.95, 0.99]
results_summary = []

feat_imp_cumsum = feat_imp.cumsum()

for th in thresholds:
    selected_features = feat_imp_cumsum[feat_imp_cumsum <= th].index.tolist()
    print(th,selected_features)
    print(f"\n===== Testing with top {th*100:.0f}% feature importance =====")
    print(f"Selected features: {len(selected_features)}")

    X_train_sel = X_train_full[selected_features]
    X_test_sel = X_test[selected_features]
    X_blind_sel = X_blind[selected_features]

    model_sel = xgb.XGBClassifier(eval_metric='mlogloss', objective='multi:softprob')
    model_sel.fit(X_train_sel.values, y_train_full.values)

    # Test set
    y_pred_test_sel = model_sel.predict(X_test_sel.values)
    acc_test = accuracy_score(y_test, y_pred_test_sel)
    f1_test = f1_score(y_test, y_pred_test_sel, average='weighted')

    # Blind set
    y_pred_blind_sel = model_sel.predict(X_blind_sel.values)
    acc_blind = accuracy_score(y_blind, y_pred_blind_sel)
    f1_blind = f1_score(y_blind, y_pred_blind_sel, average='weighted')

    print(f"Test Accuracy: {acc_test:.4f} | F1: {f1_test:.4f}")
    print(f"Blind Accuracy: {acc_blind:.4f} | F1: {f1_blind:.4f}")
    report_test = classification_report(y_test, y_pred_test_sel)
    report_blind = classification_report(y_blind, y_pred_blind_sel)
    print(f"\n Number of Features for this model: {X_test_sel.shape[1]}")
    print(f"\n📊 Classification Test Report: K2 Complete Model with {th*100:.0f}% feature importance\n", report_test)
    print(f"\n📊 Classification Blind Report: K2 Complete Model with {th*100:.0f}% feature importance\n", report_blind)

    results_summary.append({
        'Threshold': f'{int(th*100)}%',
        'N_Features': len(selected_features),
        'Test_Accuracy': acc_test,
        'Test_F1': f1_test,
        'Blind_Accuracy': acc_blind,
        'Blind_F1': f1_blind
    })

    # Salvar modelo em formatos PKL e JSON
    #joblib.dump(model_sel, f'xgb_model_{int(th*100)}perc_features.pkl')
    model_sel.save_model(f'models/xgb_model_complete_{int(th*100)}perc_features.model')

# ==============================
# SUMÁRIO FINAL
# ==============================

results_df = pd.DataFrame(results_summary)
print("\n=== Summary of Reduced Feature Tests ===")
display(results_df)
results_df.to_csv("summary/K2_complete_feature_importance_summary.csv", index=False)

end = time()
print(f"\nTotal runtime: {end - start:.2f} seconds")


Train shape: (2787, 70)
Test shape: (796, 70)
Blind validation shape: (399, 70)

Fold 1
Validation Accuracy: 0.9588 | F1-score: 0.9563

Fold 2
Validation Accuracy: 0.9659 | F1-score: 0.9622

Fold 3
Validation Accuracy: 0.9659 | F1-score: 0.9640

Fold 4
Validation Accuracy: 0.9641 | F1-score: 0.9621

Fold 5
Validation Accuracy: 0.9605 | F1-score: 0.9576

Média Validation Accuracy: 0.9630
Média Validation F1-score: 0.9604

Top 10 Feature importances:
sy_pnum            0.556319
pl_radelim         0.114011
pl_radjerr1        0.043697
default_flag       0.025850
st_meterr1         0.019346
st_met             0.014458
sy_gaiamagerr1     0.011962
soltype_encoded    0.010631
pl_radeerr1        0.010258
st_loggerr2        0.008800
dtype: float32

--- Base Model Test Results ---
Accuracy: 0.9724
F1-score: 0.9713

--- Base Model Blind Results ---
Accuracy: 0.9599
F1-score: 0.9584
0.9 ['sy_pnum', 'pl_radelim', 'pl_radjerr1', 'default_flag', 'st_meterr1', 'st_met', 'sy_gaiamagerr1', 'soltype_encod

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1028: UserWarning: [01:29:53] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)


Test Accuracy: 0.9774 | F1: 0.9765
Blind Accuracy: 0.9574 | F1: 0.9548

 Number of Features for this model: 33

📊 Classification Test Report: K2 Complete Model with 95% feature importance
               precision    recall  f1-score   support

           0       0.95      0.99      0.97       274
           1       1.00      1.00      1.00       463
           2       0.94      0.76      0.84        59

    accuracy                           0.98       796
   macro avg       0.96      0.92      0.94       796
weighted avg       0.98      0.98      0.98       796


📊 Classification Blind Report: K2 Complete Model with 95% feature importance
               precision    recall  f1-score   support

           0       0.92      0.96      0.94       138
           1       0.98      1.00      0.99       232
           2       0.90      0.62      0.73        29

    accuracy                           0.96       399
   macro avg       0.94      0.86      0.89       399
weighted avg       0.96  

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1028: UserWarning: [01:29:54] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)


Test Accuracy: 0.9736 | F1: 0.9729
Blind Accuracy: 0.9599 | F1: 0.9578

 Number of Features for this model: 46

📊 Classification Test Report: K2 Complete Model with 99% feature importance
               precision    recall  f1-score   support

           0       0.95      0.97      0.96       274
           1       1.00      1.00      1.00       463
           2       0.88      0.76      0.82        59

    accuracy                           0.97       796
   macro avg       0.94      0.91      0.93       796
weighted avg       0.97      0.97      0.97       796


📊 Classification Blind Report: K2 Complete Model with 99% feature importance
               precision    recall  f1-score   support

           0       0.93      0.96      0.94       138
           1       0.98      1.00      0.99       232
           2       0.90      0.66      0.76        29

    accuracy                           0.96       399
   macro avg       0.94      0.87      0.90       399
weighted avg       0.96  

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1028: UserWarning: [01:29:54] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)


,Threshold,N_Features,Test_Accuracy,Test_F1,Blind_Accuracy,Blind_F1
0,90%,22,0.977387,0.976997,0.954887,0.952445
1,95%,33,0.977387,0.976492,0.957393,0.954823
2,99%,46,0.973618,0.972871,0.959900,0.957823



Total runtime: 5.99 seconds


In [37]:
X_train_full.to_csv('./data/k2/X_train_full.csv',index=False)
X_test.to_csv('./data/k2/X_test.csv',index=False)
X_blind.to_csv('./data/k2/X_blind.csv',index=False)

y_train_full.to_csv('./data/k2/y_train_full.csv',index=False)
y_test.to_csv('./data/k2/y_test.csv',index=False)
y_blind.to_csv('./data/k2/y_blind.csv',index=False)